In [1]:
import ee
import os
import numpy as np
import pandas as pd
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10809'
ee.Initialize()

## city by city 

In [2]:
df = pd.read_csv("centers_reduce.csv",index_col = "Unnamed: 0")
df.head()

,NAMEASCII,POP_MAX,SOV0NAME,wof_id
0,Tokyo,35676000,Japan,102031307
1,New York,19040000,United States,85977539
2,Mexico City,19028000,Mexico,857683023
3,Mumbai,18978000,India,102030609
4,Sao Paulo,18845000,Brazil,1141909435


In [ ]:
## 0 tokyo

In [111]:
k = 8
city_name = df.iloc[k,0]
coun_name = df.iloc[k,2]
fid = int(df.iloc[k,3])
coun_name,city_name,fid

('Bangladesh', 'Dhaka', 421181477)

In [112]:
center = ee.FeatureCollection("users/zhouzz400/Boundries/city_center").filter(ee.Filter.eq("wof_id",fid)).geometry()
region = center.buffer(31000)
GAIA = ee.ImageCollection("users/zhouzz400/GAIA").filterBounds(region).mosaic()
dis_list = ee.List.sequence(1000,30000,1000)

In [113]:
center.getInfo()

{'type': 'Point', 'coordinates': [90.4066329094275, 23.72500388294307]}

### Urban res

In [114]:
yDic = ee.List([34,29,24,19,14,9,4,1])
def urbanL(yIndex):
    yIndex = ee.Number(yIndex)
    GAIA_year = GAIA.gte(yIndex)
    #dis_list = ee.List.sequence(1000,30000,1000)
    def getUrban(dis):
        buffer = center.buffer(dis)
        buffer_urban = GAIA_year.eq(1).clip(buffer)
        area_imag = buffer_urban.multiply(ee.Image.pixelArea());
        sumarea = ee.Number(area_imag.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("b1") ).round()
        return sumarea
    areaA_urban = dis_list.map(getUrban)
    return areaA_urban
##res = getYL(34)
resU= yDic.map(urbanL).getInfo()

EEException: Too many concurrent aggregations.

In [115]:
resU = []
yDic = [34,29,24,19,14,9,4,1]
for yIndex in yDic:
    yIndex = ee.Number(yIndex)
    GAIA_year = GAIA.gte(yIndex)
    def getUrban(dis):
        buffer = center.buffer(dis)
        buffer_urban = GAIA_year.eq(1).clip(buffer)
        area_imag = buffer_urban.multiply(ee.Image.pixelArea());
        sumarea = ee.Number(area_imag.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("b1") ).round()
        return sumarea
    areaA_urban = dis_list.map(getUrban)
    resU.append( areaA_urban.getInfo())

### water res

In [116]:
resW = []
yDic2 = [1985,1990,1995,2000,2005,2010,2015,2018]
for yIndex in yDic2:
    year = ee.Number(yIndex)
    water = ee.ImageCollection("JRC/GSW1_1/YearlyHistory").filter(ee.Filter.eq("year",year)).first().neq(1)
    #dis_list = ee.List.sequence(1000,30000,1000)
    def getWater(dis):
        buffer = center.buffer(dis);
        buffer_urban = water.eq(1).clip(buffer)
        area_imag = buffer_urban.multiply(ee.Image.pixelArea())
        sumarea = ee.Number(area_imag.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("waterClass") ).round()
        return sumarea
    areaA_water = dis_list.map(getWater)
    resW.append(areaA_water.getInfo())

In [ ]:
#     yIndex = ee.Number(yIndex).format()
#     year = ee.Number(year_dic.get(yIndex)) ## 
yDic2 = ee.List([1985,1990,1995,2000,2005,2010,2015,2018])
def waterL(yIndex):
    year = ee.Number(yIndex)
    water = ee.ImageCollection("JRC/GSW1_1/YearlyHistory").filter(ee.Filter.eq("year",year)).first().neq(1)
    #dis_list = ee.List.sequence(1000,30000,1000)
    def getWater(dis):
        buffer = center.buffer(dis);
        buffer_urban = water.eq(1).clip(buffer)
        area_imag = buffer_urban.multiply(ee.Image.pixelArea())
        sumarea = ee.Number(area_imag.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("waterClass") ).round()
        return sumarea
    areaA_water = dis_list.map(getWater)
    return areaA_water
resW= yDic2.map(waterL).getInfo()

In [22]:
def totoalL(yIndex):
    #dis_list = ee.List.sequence(1000,30000,1000)
    def getTotal(dis):
        return center.buffer(dis).area()
    areaA_total = dis_list.map(getTotal)
    return areaA_total
resT= yDic.map(totoalL).getInfo()

### combine 

In [117]:
cols = ["Distance","Total","1985_U","1990_U","1995_U","2000_U","2005_U","2010_U","2015_U","2018_U","1985_W","1990_W","1995_W","2000_W","2005_W","2010_W","2015_W","2018_W","1985_D","1990_D","1995_D","2000_D","2005_D","2010_D","2015_D","2018_D"]
df_r = pd.DataFrame(columns=cols)

def interSubt(list1):
    list2 = [0] +list1[:-1]
    delta = [list1[i]-list2[i] for i in range(0,len(list1))]
    return delta;

df_r.iloc[:,0]=[(x+1)*1000 for x in range(30)]
df_r.iloc[:,1]=interSubt(resT[0])
for i in range(8):
    df_r.iloc[:,i+2]=interSubt(resU[i])
    df_r.iloc[:,i+10]=interSubt(resW[i])
    for j in range(30):
        df_r.iloc[j,i+18] = df_r.iloc[j,i+2]/(df_r.iloc[j,1]-df_r.iloc[j,i+10])
df_r

,Distance,Total,1985_U,1990_U,1995_U,2000_U,2005_U,2010_U,2015_U,2018_U,...,2015_W,2018_W,1985_D,1990_D,1995_D,2000_D,2005_D,2010_D,2015_D,2018_D
0,1000,3.104061e+06,488177,1308887,1707239,2092194,2231994,2467992,2645128,2746442,...,9736,8096,0.157807,0.422515,0.551002,0.67491,0.720961,0.797191,0.854832,0.887104
1,2000,9.312182e+06,1659218,3810274,4631584,5581843,6100003,7026642,7627468,7922873,...,201896,205614,0.182257,0.418157,0.507912,0.612113,0.669417,0.770895,0.837237,0.870017
2,3000,1.552030e+07,1686063,4206725,5224685,7220145,9011636,11617340,13347750,13856944,...,750014,735837,0.114402,0.285138,0.353843,0.488564,0.610273,0.785362,0.903689,0.937264
3,4000,2.172842e+07,1281737,4108456,4606470,6171898,8343405,12373996,15957011,16866621,...,1011535,926553,0.0656009,0.207876,0.230964,0.299946,0.41367,0.603897,0.770242,0.810822
4,5000,2.793654e+07,1117920,4137646,4520624,5844200,7712830,11473562,15462333,16515721,...,1748731,1513220,0.0528353,0.186265,0.203566,0.238494,0.313045,0.457882,0.59044,0.625043
5,6000,3.414466e+07,1304007,4076758,4325632,5219848,6408696,8841003,11894034,12691224,...,3963164,2882643,0.0650063,0.179154,0.196972,0.19342,0.230752,0.30858,0.394084,0.405963
6,7000,4.035278e+07,1576609,4157183,4379460,5549425,6574290,8434503,11144152,12045485,...,4890975,3278642,0.0687455,0.159074,0.177956,0.18085,0.210806,0.249469,0.314258,0.324903
7,8000,4.656090e+07,1248662,3878370,4026220,4905053,5937410,8484279,11330640,12240257,...,5542492,3936726,0.0468984,0.129193,0.145488,0.138467,0.165262,0.22096,0.276233,0.287167
8,9000,5.276902e+07,1842195,4788432,4902420,5797269,6427626,8455457,11395939,12488871,...,8137057,6350284,0.0560213,0.135348,0.143502,0.148137,0.165617,0.202665,0.255331,0.269048
9,10000,5.897713e+07,1773006,4603843,4796663,6418635,7401772,9807763,13028042,14287521,...,7637951,5032616,0.0495679,0.117065,0.12677,0.151006,0.172354,0.210183,0.253764,0.264856


In [118]:
name ="./RES/"+str(k)+"_"+coun_name+"_"+ city_name+".csv"
df_r.to_csv(name)

## centers map 

In [10]:
country  = "Japan"
year = "1996"
index = "0_10"
coun_boun = ee.FeatureCollection("users/zhouzz400/Boundries/ne_10m_admin_0_countries").filter(ee.Filter.eq("NAME",country)).geometry()

water = ee.Image("JRC/GSW1_1/YearlyHistory/1996").neq(1)
GAIA = ee.ImageCollection("users/zhouzz400/GAIA").filterBounds(coun_boun).mosaic()
GAIA_year = GAIA.gte(19)

centers= ee.FeatureCollection("users/zhouzz400/Boundries/city_center").filterBounds(coun_boun)
centersL = centers.toList(4000).slice(0,5)

def fun1(cen):
    center = ee.Feature(cen).geometry()
    dis_list = ee.List.sequence(1000,30000,1000)
    def getUrban(dis):
        buffer = center.buffer(dis)
        buffer_urban = GAIA_year.eq(1).clip(buffer)
        area_imag = buffer_urban.multiply(ee.Image.pixelArea());
        sumarea = ee.Number(area_imag.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("b1") )
        return sumarea
    areaA_urban = dis_list.map(getUrban)
    return areaA_urban
Lurban = centersL.map(fun1)

In [11]:
Lurban.getInfo()

[[2652497.2001369116,
  10103788.347559324,
  19963966.06784596,
  31421767.254338518,
  45828777.36386361,
  60151787.74387924,
  77419985.84653391,
  93399851.84628472,
  108628746.61370103,
  121361978.06443182,
  131192165.71930271,
  138035208.76280165,
  143965645.79967707,
  148146555.99178514,
  152522099.10793507,
  156816812.497966,
  160111193.99539867,
  162606499.92472377,
  164551345.83821735,
  166906171.95283753,
  169591308.60544673,
  171484506.51671526,
  173536959.4414788,
  175729389.4145273,
  179067439.34403816,
  184217539.88572258,
  190569041.67871356,
  195875975.04596114,
  200899144.83191037,
  204133493.94912207],
 [2727038.874769984,
  11340276.36017181,
  25494029.016400285,
  44156574.25075423,
  67717979.81173524,
  93576241.31807688,
  119569723.10563089,
  150290572.54193735,
  182896765.4039393,
  214019053.8916511,
  240151604.28356007,
  264063238.28594518,
  283987876.4614301,
  306962301.7706622,
  329398443.3881791,
  349145647.44163793,
  3712